In [1]:
# 필요한 라이브러리 불러오기

from konlpy.tag import Mecab 

import pandas as pd
import numpy as np

In [12]:
# mecab 실행

mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic") # dictation path 설정 / mecab-ko-dic이 mecab의 단어사전임

In [15]:
# 형태소 분석할 데이터 준비 - 정책 내용 데이터 불러오기 

content = pd.read_csv('using_data/policy_content.csv', encoding = 'euc-kr')

In [16]:
content.head()

index            name                                        description  \
0      0        국민취업지원제도  한국형 실업부조로 고용안전망 사각지대에 있는 취업취약계층에게 취업지원서비스 및 생활...   
1      1  청년추가고용장려금 지원사업  청년을 정규직으로 추가로 고용한 중소?중견기업에 인건비를 지원함으로써, 양질의 청년...   
2      3        국민내일배움카드  국민 누구나 일자리에 도움이 되는 훈련을 받을 수 있는 평생능력개발 기반 마련을 위...   
3      4        청년내일채움공제  중소·중견기업 청년 근로자의 장기근속과 자산형성을 지원하기 위해 청년, 기업, 정부...   
4      5          청년고용지원                                          청년고용정책 지원   

                                             content  \
0  1. 취업취약계층에 취업지원\n- 취업지원서비스를 종합적으로 제공하고, 저소득 구직...   
1  1. 청년 추가채용 1명당 연 최대 900만원을 3년간 지원2. 고용위기 지역 지정...   
2  1. 국민 누구나 국민내일배움카드 신청 가능\n- 분리 운영되었던 실업자/재직자 내...   
3  1. 청년 지원내용\n - 청년 본인이 2년간 300만원(매월 12만 5천원)을 적...   
4  1. 지원내용 청년고용정책 심층 모니터링, 대학일자리센터 컨설팅 및 운영지원, 청년...   

                                                 URL  
0  https://www.youthcenter.go.kr/youngPlcyUnif/yo...  
1  https://www.youthcenter.go.kr/youngPlcyUnif/yo...  
2  https://www.youthcenter.go.kr/youngPlcyUnif/yo...  
3  https://www.youthcenter.go.kr/youngPlcyUnif/yo...  
4  https://www.youthcenter.go.kr/youngPlcyUnif/yo...

#### nouns_list  
' description ' 에서 형태소 분석처리한 모든 단어 모음 (리스트)  
' description ' 의 모든 단어를 다 보고 싶을 때 사용
     
     
#### policy_nouns  
' 한 정책 ' 의 인덱스 + 형태소 분석처리한 단어 리스트의 모음 (리스트) (DF)  
' 한 정책 ' 에 어떤 단어가 들어있는지 볼 때 사용  
  

In [17]:
nouns_list = []
policy_index = []
policy_nouns = []

for i in range(len(content['description'])):
    nouns_list.extend(mecab.nouns(content['description'][i]))
    policy_index.append(content['index'][i])
    policy_nouns.append(mecab.nouns(content['description'][i]))

policy_nouns = pd.DataFrame({'index': policy_index,
                             'nouns': policy_nouns})

### 검색단어와 앞, 뒤 2단어를 추출하여 살펴보기

In [ ]:
index_num = []
front_back_list = []

###### 단어를 입력하시오.##########
word = '지원'
###################################

for i in range(len(policy_nouns)):
    
    # 정책 리스트에 이 단어가 있으면
    if word in policy_nouns['nouns'][i]:
        
        # 검색단어가 제일 첫번째 순서면 실행
        if policy_nouns['nouns'][i].index(word) == 0:
            index_num.append(policy_nouns['index'][i])
            front_back_list.append(policy_nouns['nouns'][i][:3]) # 검색단어 + 뒷2단어
            
        # 검색단어가 두번째 순서면 실행    
        elif policy_nouns['nouns'][i].index(word) == 1:
            index_num.append(policy_nouns['index'][i])
            front_back_list.append(policy_nouns['nouns'][i][0:4])
            
        else:
            index_num.append(policy_nouns['index'][i])
            front_back_list.append(policy_nouns['nouns'][i][policy_nouns['nouns'][i].index(word)-2:policy_nouns['nouns'][i].index(word)+3])

df = pd.DataFrame()
df['index'] = index_num
df['words'] = front_back_list

### 한 글자로 이루어진 단어 추출

In [ ]:
one_word = [i for i in nouns_list if len(i) <= 2]

from collections import Counter

cnt_rank = pd.DataFrame()

cnt_rank['word'] = pd.Series(list(dict(Counter(one_word)).keys()))
cnt_rank['cnt'] = pd.Series(list(dict(Counter(one_word)).values()))

cnt_rank.sort_values(by='cnt', ascending = False)

###  모든 단어들의 빈도수 추출

In [ ]:
from collections import Counter

cnt_rank = pd.DataFrame()

cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_list)).keys()))
cnt_rank['cnt'] = pd.Series(list(dict(Counter(nouns_list)).values()))

cnt_rank.sort_values(by='cnt', ascending = True) # 내림차순으로 정렬

In [ ]:
cnt_rank.loc[cnt_rank['description_word']=='물류']

In [ ]:
one_word = [i for i in nouns_list if len(i) > 1]

from collections import Counter

cnt_rank = pd.DataFrame()

cnt_rank['word'] = pd.Series(list(dict(Counter(one_word)).keys()))
cnt_rank['cnt'] = pd.Series(list(dict(Counter(one_word)).values()))

cnt_rank.sort_values(by='cnt', ascending = False)

### 단순하게 빈도수로 테스트

In [60]:
# 사용자 글 입력
text = input()

물류, 유통 분야에 취업을 희망하는데 관련된 직무 교육 사업이 있나요?


In [61]:
words = [i for i in mecab.nouns(text) if len(i) > 1]
words

['물류', '유통', '분야', '취업', '희망', '관련', '직무', '교육', '사업']

In [ ]:
score_list = []

for i in policy_nouns['nouns']:
    score = 0
    for j in words:
        if j in i:
            score += 1
    score_list.append(score)

example = pd.DataFrame()
example['score'] = score_list

example.sort_values(by='score', ascending = False)

In [ ]:
content.iloc[example.loc[example['score']  ==  max(example['score'])].index]

In [ ]:
mecab.nouns(content.loc[content['index']==1027]['description'].iloc[0])

In [ ]:
content.loc[content['index']==1027]['description'].iloc[0]